In [1]:
import pandas as pd
from pyarrow import csv
import numpy as np
import networkx as nx
from functools import reduce
#import sweetviz as sv
import matplotlib.pyplot as plt

import timeit
import time

In [2]:
trainData_file=csv.read_csv('Data/trainData_final_nolinks_day26.csv')
training_df2 =  trainData_file.to_pandas()
training_df2

,Source,Destination,JaccardIndex,ResourceAllocation,AdamicAdar,PreferentialAttachment,CommonNeighborCentrality,Label,in_degree,out_degree,counts,max_duration,min_duration,avg_duration,max_SrcBytes,min_SrcBytes,avg_SrcBytes,max_DstBytes,min_DstBytes,avg_DstBytes
0,ActiveDirectory,Comp000566,0.000993,0.106979,1.570389,140924,5206.4,Missing,4,3197,17.0,243335.0,13949.0,126359.117647,22656.0,1248.0,11796.705882,0.0,0.0,0.0
1,ActiveDirectory,Comp001022,0.001191,0.008405,1.494416,221452,5208.0,Missing,4,3197,3.0,1.0,1.0,1.000000,96.0,96.0,96.000000,0.0,0.0,0.0
2,ActiveDirectory,Comp002524,0.002281,0.009277,2.840069,412706,5216.8,Missing,9,3197,1.0,1.0,1.0,1.000000,96.0,96.0,96.000000,0.0,0.0,0.0
3,ActiveDirectory,Comp003688,0.001786,0.006173,2.173534,281848,5212.8,Missing,2,3197,1.0,26.0,26.0,26.000000,0.0,0.0,0.000000,2287.0,2287.0,2287.0
4,ActiveDirectory,Comp005295,0.002776,0.363384,5.377657,483168,5220.8,Missing,13,3197,2.0,984.0,86.0,535.000000,0.0,0.0,0.000000,2471.0,2411.0,2441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366347,Comp273343,Comp017911,0.000000,0.000000,0.000000,252,1732.8,NoLink,9,28,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
366348,Comp852778,Comp987487,0.078947,0.000351,0.331175,420,2601.6,NoLink,2,19,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
366349,Comp010016,Comp284265,0.050000,0.000088,0.107098,38,2600.0,NoLink,0,19,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
366350,Comp776638,Comp428151,0.384615,0.002573,1.186201,323,2607.2,NoLink,4,15,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [3]:
testData_file=csv.read_csv('Data/testData_final_nolinks_day26.csv')
testing_df2 =  testData_file.to_pandas()
testing_df2

,Source,Destination,JaccardIndex,ResourceAllocation,AdamicAdar,PreferentialAttachment,CommonNeighborCentrality,Label,in_degree,out_degree,counts,max_duration,min_duration,avg_duration,max_SrcBytes,min_SrcBytes,avg_SrcBytes,max_DstBytes,min_DstBytes,avg_DstBytes
0,ActiveDirectory,Comp003448,0.001699,0.020821,1.893824,172935,5247.6,Missing,5,2025,1.0,28.0,28.0,28.0,0.0,0.0,0.0,2255.0,2255.0,2255.0
1,ActiveDirectory,Comp004355,0.001092,0.001481,1.029108,115290,5243.6,Missing,1,2025,2.0,1.0,1.0,1.0,60.0,60.0,60.0,0.0,0.0,0.0
2,ActiveDirectory,Comp009410,0.002063,0.038536,2.493647,181170,5250.0,Missing,3,2025,1.0,4.0,4.0,4.0,640.0,640.0,640.0,0.0,0.0,0.0
3,ActiveDirectory,Comp014675,0.001578,0.002518,1.511377,123525,5246.8,Missing,1,2025,3.0,29642.0,16122.0,22432.0,1740.0,1020.0,1280.0,0.0,0.0,0.0
4,ActiveDirectory,Comp014941,0.001092,0.003264,1.079179,98820,5243.6,Missing,1,2025,1.0,1.0,1.0,1.0,309.0,309.0,309.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243997,Comp050085,Comp972195,0.000000,0.000000,0.000000,0,0.0,NoLink,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243998,Comp114983,Comp372203,0.333333,0.000199,0.117335,3,2619.0,NoLink,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243999,Comp639223,Comp158414,0.000000,0.000000,0.000000,68,1309.1,NoLink,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244000,Comp155068,Comp382098,0.391304,0.002147,1.061073,256,2625.4,NoLink,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_test, y_test = testing_df2[['JaccardIndex', 'ResourceAllocation','AdamicAdar','PreferentialAttachment', 'CommonNeighborCentrality','in_degree','out_degree']], testing_df2['Label']

model = RandomForestClassifier()
#model.fit(X_train, y_train)

#print(f"Accuracy on the test set: {accuracy_score(y_test, model.predict(X_test))}")

In [5]:
len(training_df2[training_df2["Label"]=="Continuing"])


153676

In [6]:
len(training_df2[training_df2["Label"]=="Missing"])


156371

In [7]:
len(training_df2[training_df2["Label"]=="New"])

36309

In [8]:
len(training_df2[training_df2["Label"]=="NoLink"])

19996

In [9]:
len(testing_df2[testing_df2["Label"]=="Continuing"])

145250

In [10]:
len(testing_df2[testing_df2["Label"]=="Missing"])

44735

In [11]:
len(testing_df2[testing_df2["Label"]=="New"])

34020

In [12]:
TrainTest=[]
TrainTest.append(training_df2)
TrainTest.append(testing_df2)

In [13]:
trainData=training_df2.copy()

In [14]:
train_Cont_df=trainData[trainData["Label"]=="Continuing"]
train_Miss_df=trainData[trainData["Label"]=="Missing"]
train_New_df=trainData[trainData["Label"]=="New"]
train_NoLink_df=trainData[trainData["Label"]=="NoLink"]

In [15]:
# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLink']
#report_2 = classification_report(y_test, pred, target_names=target_names)
#print(f"Test Score\t {pred}")
#print(report_2)

In [16]:
dfc=train_Cont_df.sample(n = 19000)
dfm=train_Miss_df.sample(n = 19000)
dfn=train_New_df.sample(n = 19000)
dfnl=train_NoLink_df.sample(n = 19000)

In [17]:
trainData_df3=pd.concat([dfc,dfm,dfn,dfnl],ignore_index=True)
X_train_3, y_train_3 = trainData_df3[['JaccardIndex', 'ResourceAllocation','AdamicAdar','PreferentialAttachment', 'CommonNeighborCentrality','in_degree','out_degree']], trainData_df3['Label']


In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score


clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train_3, y_train_3)
pred_adaboost = clf.predict(X_test)
Adaboost_Accuracy_score=accuracy_score(y_test, pred_adaboost)

print(f"Accuracy on the test set: {Adaboost_Accuracy_score}")

# Step 5 Performance Evaluation: Confusion Matrix
from sklearn.metrics import confusion_matrix
#print("Sample size for Computer Work   =",(X['Activity']==2).sum())

pred_adaboost = clf.predict(X_test)
cm_adaboost = confusion_matrix(y_test, pred_adaboost)

print('Confusion Matrix: \n', cm_adaboost)

# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLinks']
report_2 = classification_report(y_test, pred_adaboost, target_names=target_names)
#print(f"Test Score\t {pred}")
print(report_2)

Accuracy on the test set: 0.49011483512430226
Confusion Matrix: 
 [[60351 77252  7647     0]
 [13620 27306  3809     0]
 [   99   610 12756 20555]
 [    2     2   817 19176]]
              precision    recall  f1-score   support

  Continuing       0.81      0.42      0.55    145250
     Missing       0.26      0.61      0.36     44735
         New       0.51      0.37      0.43     34020
     NoLinks       0.48      0.96      0.64     19997

    accuracy                           0.49    244002
   macro avg       0.52      0.59      0.50    244002
weighted avg       0.64      0.49      0.51    244002



In [19]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_3, y_train_3)
pred_gnb = gnb.predict(X_test)

gnb_Accuracy_score=accuracy_score(y_test, pred_gnb)

print(f"Accuracy on the test set with Guassian Naive Baye: {gnb_Accuracy_score}")
# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLinks']
report_2 = classification_report(y_test, pred_gnb, target_names=target_names)
#print(f"Test Score\t {pred}")
print(report_2)

Accuracy on the test set with Guassian Naive Baye: 0.3900131966131425
              precision    recall  f1-score   support

  Continuing       0.63      0.04      0.08    145250
     Missing       0.23      0.92      0.36     44735
         New       1.00      0.82      0.90     34020
     NoLinks       0.81      1.00      0.89     19997

    accuracy                           0.39    244002
   macro avg       0.67      0.69      0.56    244002
weighted avg       0.62      0.39      0.31    244002



In [20]:
from sklearn.ensemble import GradientBoostingClassifier

XGB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
XGB.fit(X_train_3, y_train_3)
pred_XGB = XGB.predict(X_test)

XGB_Accuracy_score=accuracy_score(y_test, pred_XGB)

print(f"Accuracy on the test set with Gradient_Boosting_Classifier : {XGB_Accuracy_score}")

# Step 5 Performance Evaluation: Confusion Matrix
from sklearn.metrics import confusion_matrix


pred_XGB = XGB.predict(X_test)
cm_XGB = confusion_matrix(y_test, pred_XGB)

print('Confusion Matrix: \n', cm_XGB)

# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLinks']
report_2 = classification_report(y_test, pred_XGB, target_names=target_names)
#print(f"Test Score\t {pred}")
print(report_2)

Accuracy on the test set with Gradient_Boosting_Classifier : 0.42643912754813484
Confusion Matrix: 
 [[ 15048 130080      0    122]
 [   896  43811      0     28]
 [    18   1577  31029   1396]
 [     0      9   5824  14164]]
              precision    recall  f1-score   support

  Continuing       0.94      0.10      0.19    145250
     Missing       0.25      0.98      0.40     44735
         New       0.84      0.91      0.88     34020
     NoLinks       0.90      0.71      0.79     19997

    accuracy                           0.43    244002
   macro avg       0.73      0.68      0.56    244002
weighted avg       0.80      0.43      0.37    244002



In [21]:
#import svm model
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

LinearSVC  = make_pipeline(StandardScaler(),LinearSVC(dual=False, random_state=0, tol=1e-5))
LinearSVC.fit(X_train_3, y_train_3)
pred_LinearSVC = LinearSVC.predict(X_test)

LinearSVC_Accuracy_score=accuracy_score(y_test, pred_LinearSVC)

print(f"Accuracy on the test set with LinearSVC : {LinearSVC_Accuracy_score}")

# Step 5 Performance Evaluation: Confusion Matrix
from sklearn.metrics import confusion_matrix


pred_LinearSVC= LinearSVC.predict(X_test)
cm_LinearSVC = confusion_matrix(y_test, pred_LinearSVC)

print('Confusion Matrix: \n', cm_LinearSVC)

# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLinks']
report_2 = classification_report(y_test, pred_LinearSVC, target_names=target_names)
#print(f"Test Score\t {pred}")
print(report_2)

Accuracy on the test set with LinearSVC : 0.6224129310415488
Confusion Matrix: 
 [[74839 70319     0    92]
 [15911 28809     0    15]
 [   38  1568 28306  4108]
 [    0     9    72 19916]]
              precision    recall  f1-score   support

  Continuing       0.82      0.52      0.63    145250
     Missing       0.29      0.64      0.40     44735
         New       1.00      0.83      0.91     34020
     NoLinks       0.83      1.00      0.90     19997

    accuracy                           0.62    244002
   macro avg       0.73      0.75      0.71    244002
weighted avg       0.75      0.62      0.65    244002



In [22]:
#import Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LogReg = LogisticRegression(random_state=0)
LogReg.fit(X_train_3, y_train_3)
pred_LogReg = LogReg.predict(X_test)

LogReg_Accuracy_score=accuracy_score(y_test, pred_LogReg)

print(f"Accuracy on the test set with LogReg : {LogReg_Accuracy_score}")

# Step 5 Performance Evaluation: Confusion Matrix
from sklearn.metrics import confusion_matrix


pred_LogReg= LogReg.predict(X_test)
cm_LogReg = confusion_matrix(y_test, pred_LogReg)

print('Confusion Matrix: \n', cm_LogReg)

# Step 5 Performance Evaluation: 
from sklearn.metrics import classification_report, confusion_matrix

target_names=['Continuing','Missing', 'New','NoLinks']
report_2 = classification_report(y_test, pred_LogReg, target_names=target_names)
#print(f"Test Score\t {pred}")
print(report_2)

/home/asha/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy on the test set with LogReg : 0.4800288522225228
Confusion Matrix: 
 [[73283 58314     0 13653]
 [12990 27199     0  4546]
 [ 9236 21961   530  2293]
 [    1   158  3722 16116]]
              precision    recall  f1-score   support

  Continuing       0.77      0.50      0.61    145250
     Missing       0.25      0.61      0.36     44735
         New       0.12      0.02      0.03     34020
     NoLinks       0.44      0.81      0.57     19997

    accuracy                           0.48    244002
   macro avg       0.40      0.48      0.39    244002
weighted avg       0.56      0.48      0.48    244002

